# 【Pandas】groupby/pivot_tableでのカテゴリー別集計方法を解説

In [43]:
#Kaggle-> https://www.kaggle.com/c/titanic
#タイタニックデータセット (Titanic - Machine Learning from Disaster)
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer

# データフレームのセット
df_taitanic = pd.read_csv("../Data/taitanic.csv")
Column_List = df_taitanic.columns

# データ形状の確認
print('--------------------------------------------------')
print('df_taitanic_shape: (%i,%i)' % df_taitanic.shape)
print('--------------------------------------------------')

df_taitanic.head()

--------------------------------------------------
df_taitanic_shape: (891,12)
--------------------------------------------------


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 1. データフレームの基礎情報

In [20]:
#describeで各カラムの情報の集計
df_taitanic.describe(include="all")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Parkes, Mr. Francis ""Frank""",male,NaN,NaN,NaN,1601,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [22]:
#Columnの情報抽出
print(df_taitanic.columns)

#indexの情報抽出
print(df_taitanic.index)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
RangeIndex(start=0, stop=891, step=1)


## 2. groupby()によるデータ整理 

In [42]:
#生存/亡くなった方の年齢平均
print(df_taitanic.groupby(["Survived"])["Age"].mean())

#生存/亡くなった方の各Pclassごとの年齢平均
print(df_taitanic.groupby(["Survived","Pclass"])["Age"].mean())

Survived
0    30.626179
1    28.343690
Name: Age, dtype: float64
Survived  Pclass
0         1         43.695312
          2         33.544444
          3         26.555556
1         1         35.368197
          2         25.901566
          3         20.646118
Name: Age, dtype: float64


In [44]:
#生存/亡くなった方の性別ごとの合計
print(df_taitanic.groupby(["Sex"])["Survived"].sum())

#生存/亡くなった方の性別/各Pclassごとの合計
print(df_taitanic.groupby(["Sex","Pclass"])["Survived"].sum())

Sex
female    233
male      109
Name: Survived, dtype: int64
Sex     Pclass
female  1         91
        2         70
        3         72
male    1         45
        2         17
        3         47
Name: Survived, dtype: int64


In [50]:
#生存/亡くなった方の年齢のmin/max/mean
print(df_taitanic.groupby(["Survived"])["Age"].agg([min,max,np.mean]))

#生存/亡くなった方の各Pclassごとの年齢min/max/mean
print(df_taitanic.groupby(["Survived","Pclass"])["Age"].agg([min,max,np.mean]))

           min   max       mean
Survived                       
0         1.00  74.0  30.626179
1         0.42  80.0  28.343690
                   min   max       mean
Survived Pclass                        
0        1        2.00  71.0  43.695312
         2       16.00  70.0  33.544444
         3        1.00  74.0  26.555556
1        1        0.92  80.0  35.368197
         2        0.67  62.0  25.901566
         3        0.42  63.0  20.646118


## 3. Pivottable()によるデータ整理

In [59]:
#ピボットテーブルでの性別別 年齢中央値の取得
print(df_taitanic.pivot_table(values= "Age" , index= "Sex", aggfunc= np.median))

#ピボットテーブルでの性別別 年齢平均値と中央値の取得
print(df_taitanic.pivot_table(values= "Age" , index= "Sex", aggfunc= [np.mean, np.median]))

         Age
Sex         
female  27.0
male    29.0
             mean median
              Age    Age
Sex                     
female  27.915709   27.0
male    30.726645   29.0


In [60]:
#ピボットテーブルでの生存者/死亡者それぞれの性別別 年齢平均値の取得
print(df_taitanic.pivot_table(values= "Age" , index= "Sex", columns="Survived", margins=True, aggfunc = np.mean))

Survived          0          1        All
Sex                                      
female    25.046875  28.847716  27.915709
male      31.618056  27.276022  30.726645
All       30.626179  28.343690  29.699118
